# REDES NEURAIS ARTIFICIAIS
## Evaluate Improve e Tunning

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2022-03-04 22:44:03.782054: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-04 22:44:03.782114: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tf.__version__

'2.8.0'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [9]:
ann = tf.keras.models.Sequential()

2022-03-04 22:45:41.015427: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-04 22:45:41.015513: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-04 22:45:41.015567: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (israel-BookX40): /proc/driver/nvidia/version does not exist
2022-03-04 22:45:41.016278: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Adding the input layer and the first hidden layer

In [10]:
ann.add(tf.keras.layers.Dense(units=6, activation='tanh'))

### Adding the second hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units=6, activation='tanh'))

### Adding the output layer

In [12]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [13]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [14]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.5922 - accuracy: 0.7036
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4415 - accuracy: 0.8071
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4223 - accuracy: 0.8129
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4156 - accuracy: 0.8148
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4084 - accuracy: 0.8166
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4007 - accuracy: 0.8216
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3930 - accuracy: 0.8244
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3864 - accuracy: 0.8264
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3802 - accuracy: 0.8303
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3757 - accura

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [15]:
y_pred = ann.predict(X_test)
y_pred

array([[0.2563545 ],
       [0.39366886],
       [0.18624768],
       ...,
       [0.15604362],
       [0.17573604],
       [0.25315064]], dtype=float32)

In [16]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1511   84]
 [ 194  211]]


---
# Avaliando o Modelo

In [19]:
# Evaluating the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [32]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [33]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

/tmp/ipykernel_90335/1297209367.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)


Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
720/720 [==============================] - 4s 5ms/step - loss: 0.4858 - accuracy: 0.8072
Epoch 2/10
720/720 [==============================] - 4s 5ms/step - loss: 0.4816 - accuracy: 0.8062
Epoch 2/10
720/720 [==============================] - 5s 5ms/step - loss: 0.4860 - accuracy: 0.8039
Epoch 2/10
720/720 [==============================] - 5s 5ms/step - loss: 0.4853 - accuracy: 0.8090
Epoch 2/10
720/720 [==============================] - 5s 6ms/step - loss: 0.4885 - accuracy: 0.8051
Epoch 2/10
720/720 [==============================] - 5s 5ms/step - loss: 0.4861 - accuracy: 0.8028
Epoch 2/10
720/720 [==============================] - 3s 5ms/step - loss: 0.4340 - accuracy: 0.8119
Epoch 3/10
720/720 [==============================] - 3s 5ms/step - loss: 0.4338 - accuracy: 0.8128
Epoch 3/10
720/720 [==============================] - 3s 4ms/step - loss: 0.4319 - accuracy: 0.8114
Epoch 3/10
720/720 [===

In [36]:
print(mean,variance)

0.8461249947547913 0.0101773472025958


# Melhorando o modelo

In [37]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from tensorflow.keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [38]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

/tmp/ipykernel_90335/1297209367.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)


Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
720/720 [==============================] - 5s 5ms/step - loss: 0.4897 - accuracy: 0.8053
Epoch 2/10
720/720 [==============================] - 5s 5ms/step - loss: 0.4895 - accuracy: 0.8074
Epoch 2/10
720/720 [==============================] - 5s 5ms/step - loss: 0.4860 - accuracy: 0.8051
Epoch 2/10
720/720 [==============================] - 5s 5ms/step - loss: 0.4962 - accuracy: 0.8044
Epoch 2/10
720/720 [==============================] - 5s 5ms/step - loss: 0.4893 - accuracy: 0.8082
Epoch 2/10
720/720 [==============================] - 3s 5ms/step - loss: 0.4363 - accuracy: 0.8117
Epoch 3/10
720/720 [==============================] - 3s 5ms/step - loss: 0.4370 - accuracy: 0.8108
Epoch 3/10
720/720 [==============================] - 3s 5ms/step - loss: 0.4350 - accuracy: 0.8125
Epoch 3/10
720/720 [==============================] - 4s 5ms/step - loss: 0.4355 - accuracy: 0.8147
Epoch 3/10
720/720 [===

In [39]:
print(mean,variance)

0.8303749918937683 0.012624996251410673


# Tuning the ANN

In [42]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

/tmp/ipykernel_90335/4010492138.py:16: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier)


In [43]:
%%time

grid_search = grid_search.fit(X_train, y_train)

Epoch 1/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4833 - accuracy: 0.8081
Epoch 2/50
720/720 [==============================] - 1s 2ms/step - loss: 0.4360 - accuracy: 0.8144
Epoch 3/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4354 - accuracy: 0.8108
Epoch 4/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4344 - accuracy: 0.8112
Epoch 5/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4346 - accuracy: 0.8138
Epoch 6/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4303 - accuracy: 0.8097
Epoch 7/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4253 - accuracy: 0.8149
Epoch 8/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4159 - accuracy: 0.8214
Epoch 9/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4078 - accuracy: 0.8279
Epoch 10/50
720/720 [==============================] - 1s 1ms/step - loss: 0.3978 - accuracy: 0.8331

In [44]:
print('Best params:\n',grid_search.best_params_,'\nBest accuracy:\n',grid_search.best_score_)

Best params:
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'adam'} 
Best accuracy:
 0.8543750000000001


# Halving Grid Search

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

halving_cv = HalvingGridSearchCV(estimator = classifier,
                                  param_grid = parameters,
                                  scoring="accuracy", 
                                  n_jobs=-1, 
                                  min_resources="exhaust", 
                                  factor=3,
                                  cv = 10)

In [ ]:
%%time
halving_cv = halving_cv.fit(X_train, y_train)

Epoch 1/100
800/800 [==============================] - 2s 1ms/step - loss: 0.5946 - accuracy: 0.7885
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4398 - accuracy: 0.7998
Epoch 3/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4418 - accuracy: 0.7957
Epoch 4/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4419 - accuracy: 0.7951
Epoch 5/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4271 - accuracy: 0.8002
Epoch 6/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4255 - accuracy: 0.8023: 0s - loss: 0.4204 - 
Epoch 7/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4353 - accuracy: 0.7938
Epoch 8/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4432 - accuracy: 0.7897
Epoch 9/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4534 - accuracy: 0.7873
Epoch 10/100
800/800 [==============================] - 1s 1ms/step -

In [ ]:
print('Best params:\n',halving_cv.best_params_,'\nBest accuracy:\n',halving_cv.best_score_)

Best params:
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'rmsprop'} 
Best accuracy:
 0.8346683354192741


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)